In [1]:
import sys
import ibm_db
import ibm_db_dbi

In [2]:
from pymongo import MongoClient
import datetime
#conecta banco
client = MongoClient('localhost', 27017)
#seleciona base
db = client.S4

#seleciona collatction
collection = db.SAC

In [3]:
from ipywidgets import FloatProgress
from IPython.display import display

In [4]:
import json
import bson
from bson import json_util
from bson.codec_options import CodecOptions
from bson.raw_bson import RawBSONDocument


In [5]:
#Enter the values for you database connection
dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = "SAC"            # e.g. "BLUDB"
dsn_hostname = "172.23.2.120" # e.g.: "awh-yp-small03.services.dal.bluemix.net"
dsn_port = "50100"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "UNJ_BI_PROCURADORIAS"        # e.g. "dash104434"
dsn_pwd = "3z@kthAzb"       # e.g. "7dBZ3jWt9xN6$o0JiX!m"

In [6]:
dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE={0};"
    "HOSTNAME={1};"
    "PORT={2};"
    "PROTOCOL=TCPIP;"
    "UID={3};"
    "PWD={4};").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)

In [7]:
conn = ibm_db.connect(dsn_database, dsn_uid, dsn_pwd) 
print("Conectou!!!")
schema = ibm_db.exec_immediate(conn, "SET SCHEMA SAC")

Conectou!!!


In [8]:
sql = """SELECT DISTINCT 
       ia.NUATENDIMENTO,
       IA.NUITEM,
       cast(ia.DEATENDIMENTO as varchar(32522)) AS DESCRICAO ,
       ve.NURELEASE,
       ia.CDLOCALSISTEMA,
       ia.CDMODULOSISTEMA,
       CLI.SGCLIENTE AS CLIENTE,
       IA.DTREGISTRO AS DATA_CADASTRO,
       IA.DTLIMSOLUCAO AS DATA_SLA,
       CASE IA.CDPRIORIDADE 
         WHEN 6 THEN 'ZERO/CRÍTICA'
         WHEN 5 THEN 'UM/IMPEDITIVA'
         WHEN 1 THEN 'DOIS/ALTA'
         WHEN 2 THEN 'TRES/MÉDIA'
         WHEN 3 THEN 'QUATRO/BAIXA'
         WHEN 4 THEN 'INDEFINIDA'
       END SEVERIDADE,
       CASE IA.CDPRIORIDADE 
         WHEN 6 THEN 8
         WHEN 5 THEN 5
         WHEN 1 THEN 4
         WHEN 2 THEN 3
         WHEN 3 THEN 2
         WHEN 4 THEN 3
       END PONTUACAO_SEVERIDADE,
       SIT.DESITUACAOATEND AS SITUACAO_SALT,
       ( SELECT (NUATENDvinc || '/' || NUITEMvinc)  FROM ESACVINCULAATEND aaa where aaa.NUATENDVINC = ia.NUATENDIMENTO AND aaa.NUITEMVINC = ia.NUITEM  FETCH FIRST 1 ROWS ONLY) AS SaltsVinculadas  
  FROM sac.ESACREGISTROATEND ra
 LEFT JOIN sac.ESACVERSAO ve
    ON ra.CDPROJETO = ve.CDPROJETO
   AND ra.CDSISTEMA = ve.CDSISTEMA
   AND ra.NUSEQVERSAO = ve.NUSEQVERSAO
  JOIN sac.ESACITEMATEND ia
    ON ia.cdProjeto = ia.CDPROJETO
   AND ia.NUATENDIMENTO = ra.NUATENDIMENTO
  JOIN SAC.ESACSITUACAOATEND SIT
    ON SIT.CDSITUACAOATEND = IA.CDSITUACAOATEND
  JOIN SAC.ESACCLIENTE CLI
    ON CLI.CDCLIENTE = ra.CDCLIENTE
 WHERE ra.cdProjeto = 3
   AND ra.CDSISTEMA IN (31,93)
   AND VARCHAR_FORMAT (ia.DTREGISTRO,'YYYY-MM-DD') >= '2014-01-01';"""

In [9]:
stmt = ibm_db.exec_immediate(conn, sql)
result = ibm_db.fetch_assoc(stmt)
total = 99999
f = FloatProgress(min=0, max=total, description="{:1.0f} tasks left ".format(total), bar_style='success')
f.value=0
display(f)  
SALTDB = dict()
while(result):
    if  (result['DATA_CADASTRO'] is not None):
        result['DATA_CADASTRO'] =  result['DATA_CADASTRO'].isoformat()
    if  (result['DATA_SLA'] is not None):
        result['DATA_SLA'] =  result['DATA_SLA'].isoformat()  
    SALTDB = json_util.dumps(result)
    SALTDB = bson.BSON.encode(result)
    f.value += 1
    f.description =  str(f.value)  + '/' + str(total)
    db.salts.insert_one(RawBSONDocument(SALTDB))
    result = ibm_db.fetch_assoc(stmt)

FloatProgress(value=0.0, bar_style='success', description='99999 tasks left ', max=99999.0)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

